In [ ]:
import os
import numpy as np
import random
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from tensorflow.keras.callbacks import EarlyStopping

# 경로 설정
pass_dir = '/content/drive/MyDrive/data/태림산업 이미지셋/SHAFT/SHAFT_합격이미지'
fail_dir = '/content/drive/MyDrive/data/태림산업 이미지셋/SHAFT/SHAFT_불량이미지'
img_height, img_width = 224, 224
batch_size = 16  # batch_size 변수 정의

# 데이터 증강 생성기 설정
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# 부스트래핑 반복 횟수
n_iterations = 4
accuracy_scores = []

for i in range(n_iterations):
    # 랜덤으로 30개 이미지 추출
    pass_images = random.sample(os.listdir(pass_dir), 30)
    fail_images = random.sample(os.listdir(fail_dir), 30)

    X, y = [], []
    for img_name in pass_images:
        img_path = os.path.join(pass_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_height, img_width))
        X.append(img)
        y.append(1)  # pass label

    for img_name in fail_images:
        img_path = os.path.join(fail_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (img_height, img_width))
        X.append(img)
        y.append(0)  # fail label

    X = np.array(X)
    y = np.array(y)

    # 데이터셋 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y)

    # 데이터 증강을 통해 학습 데이터 증가
    X_augmented, y_augmented = [], []
    for xi, yi in zip(X_train, y_train):
        xi = np.expand_dims(xi, axis=0)
        gen = datagen.flow(xi, np.array([yi]), batch_size=1)
        for _ in range(10):  # 각 이미지당 10개 증강
            augmented_x, augmented_y = next(gen)  # Python의 next() 함수 사용
            X_augmented.append(augmented_x[0])
            y_augmented.append(augmented_y[0])

    X_augmented = np.array(X_augmented)
    y_augmented = np.array(y_augmented)

    # 모델 정의
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

    # 모델 컴파일
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    # 조기 종료 설정
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # 모델 학습
    model.fit(
        X_augmented, y_augmented,
        epochs=20,
        batch_size=batch_size,  # 정의된 batch_size 사용
        validation_split=0.2,
        callbacks=[early_stopping]
    )

    # 모델 평가
    y_pred = (model.predict(X_test) > 0.5).astype(int).flatten()
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)
    print(f"Iteration {i+1}/{n_iterations} - Accuracy: {accuracy:.4f}")

# 평균 정확도 출력
average_accuracy = np.mean(accuracy_scores)
print(f"Average Accuracy over {n_iterations} iterations: {average_accuracy:.4f}")